In [ ]:
# from google.colab import userdata

# ngrok_key = userdata.get('ngrok_key')
# line_access_token = userdata.get('line_access_token')
# line_secret = userdata.get('line_secret')
# port = 5000

In [15]:
import os

line_access_token = os.environ.get('LINE_ACCESS_TOKEN')
line_secret = os.environ.get('LINE_SECRET')
open_api_key = os.environ.get('OPENAI_API_KEY')
port = 5000

In [16]:
from openai import OpenAI

client = OpenAI(
    base_url='http://120.105.97.239:11434/v1/',

    api_key='ollama',
)
def get_completion(prompt):
    messages=[
        {
            'role': 'system',
            'content': '你是一個說繁體中文的助理，只會用台灣用語，不會用中國用語',
            'role': 'user',
            'content': prompt,

        }
    ]
    response = client.chat.completions.create(
        model='jcai/llama3-taide-lx-8b-chat-alpha1:f16',
        messages=messages,

    )
    return response.choices[0].message.content


In [17]:
def summarize(text):
    messages=[
        {
            'role': 'system',
            'content': '你是一個說繁體中文的助理，只會用台灣用語，不會用中國用語, 將下列文字進行摘要，輸出字數在256個字以內',
            'role': 'user',
            'content': text,

        }
    ]
    response = client.chat.completions.create(
        model='jcai/llama3-taide-lx-8b-chat-alpha1:f16',
        messages=messages,
        max_tokens=256,

    )
    return response.choices[0].message.content
    

In [18]:
prompt = "簡介東吳大學"
output = get_completion(prompt)
print(summarize(output))

對於東吳大學這個題目，我可以提供以下資訊：
東吳大學(英語：Soochow University)，成立於1900年，是一所位於台灣新北市蘇澳區及桃園市桃園區的著名私立大學，也是台灣最早的私立大學之一。該校由美國基督教長老教會所創辦，以崇尚自由、尊重多元為校訓，提供包含人文社會科學、理學、法學、管理、公共行政、國際事務等領域的學科。
東吳以其優美的校園景觀、豐沛的學術資源、活絡的校園生活著稱。學校除了主校區外，在桃園還設有第二校區，提供更多元化的教學與研究空間。此外，該校在中國大陸上海市也設有分校區，為兩岸交流合作做出貢獻。
東吳大學以其優良的教學品質、重視人文關懷的理念，在國內外均享有盛譽，是許多台灣學生選填


In [ ]:
messages = []

In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    StickerMessage,
    LocationMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    StickerMessageContent,
    LocationMessageContent
)

app = Flask(__name__)

configuration = Configuration(access_token=line_access_token)
handler = WebhookHandler(line_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            messages.append(prompt)
            prompts = " ".join(messages)
            prompts = summarize(prompts)
            reply_text = get_completion(prompts)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )
            messages.append(reply_text)
        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

        
@handler.add(MessageEvent, message=StickerMessageContent)
def handle_sticker_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[StickerMessage(
                    package_id=event.message.package_id,
                    sticker_id=event.message.sticker_id)
                ]
            )
        )

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_location_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[LocationMessage(
                    title='定位訊息',
                    address=event.message.address,
                    latitude=event.message.latitude,
                    longitude=event.message.longitude
                )]
            )
        )

if __name__ == "__main__":
    app.run(port=port)